In [1]:
import numpy as np
import scipy as sp

In [2]:
def invd_sparse(x):
    """
    Constructs a diagonal matrix with 1/x, replacing entries where x == 0 with 1.

    Parameters:
        x (numpy array): Input 1D array.

    Returns:
        numpy array: A diagonal matrix.
    """
    with np.errstate(divide="ignore", invalid="ignore"):
        invd_values = np.where(x != 0, 1.0 / x, 1.0)
    return sp.diags(invd_values.flatten())

In [3]:
def GRAS_UpdateMatrix(P, N, r, s):
    """
    Updates the matrix using the GRAS balancing approach.
    """
    return np.diag(r.flatten()) @ P @ np.diag(s.flatten()) - invd(r) @ N @ invd(s)

In [14]:
import warnings

import scipy.sparse as sp


def invd_sparse(x):
    """
    Constructs a diagonal sparse matrix with 1/x, replacing entries where x == 0 with 1.

    Parameters:
        x (numpy array or scipy sparse matrix): Input 1D array.

    Returns:
        scipy sparse matrix: A diagonal sparse matrix.
    """
    with np.errstate(divide="ignore", invalid="ignore"):
        invd_values = np.where(x != 0, 1.0 / x, 1.0)
    return sp.diags(invd_values.flatten())


def GRAS_UpdateMatrix(P, N, r, s):
    """
    Updates the matrix using the GRAS balancing approach with sparse matrices.

    Parameters:
        P (scipy sparse matrix): Positive part of the input matrix.
        N (scipy sparse matrix): Negative part of the input matrix.
        r (numpy array): Row adjustment factors.
        s (numpy array): Column adjustment factors.

    Returns:
        scipy sparse matrix: The updated balanced matrix.
    """
    return sp.diags(r.flatten()) @ P @ sp.diags(s.flatten()) - invd_sparse(r) @ N @ invd_sparse(s)


def GRAS_Balancing(X, u, v, EPSILON=1e-10, epsilon_constr=1e-10, max_iter=20):
    # Ensure inputs are consistent
    assert np.allclose(u.sum(), v.sum()), "Row and column sums must be equal."

    # Define matrix size
    m, n = X.shape

    # Split into positive and negative components (in sparse format)
    P = sp.csr_matrix(np.maximum(X, 0))
    N = sp.csr_matrix(np.maximum(-X, 0))
    assert np.allclose(X, (P - N).toarray()), "Matrix decomposition into P and N is incorrect."

    # Initialize vectors
    r = np.ones((m, 1))
    pr = P.T @ r
    nr = N.T @ invd_sparse(r) @ np.ones((m, 1))

    # Initial value of s1
    # Ensure pr and nr are compatible for element-wise multiplication
    pr_dense = pr.toarray() if sp.issparse(pr) else pr
    nr_dense = nr.toarray() if sp.issparse(nr) else nr
    s1 = invd_sparse(2 * pr_dense) @ (v + np.sqrt(v**2 + 4 * pr_dense * nr_dense))

    # Handle possible NaNs
    s1 = np.nan_to_num(s1, nan=1e-10, posinf=1e-10, neginf=1e-10)

    ss = -invd_sparse(v) @ nr
    s1[pr_dense.flatten() == 0] = ss[pr_dense.flatten() == 0]

    # Iteration loop
    iter = 1
    dif = float("inf")

    while (dif > EPSILON) and (iter <= max_iter):
        # Update r
        ps = P @ s1
        ns = N @ invd_sparse(s1) @ np.ones((n, 1))

        ps_dense = ps.toarray() if sp.issparse(ps) else ps
        ns_dense = ns.toarray() if sp.issparse(ns) else ns
        r = invd_sparse(2 * ps_dense) @ (u + np.sqrt(u**2 + 4 * ps_dense * ns_dense))

        # Handle possible NaNs
        r = np.nan_to_num(r, nan=1e-10, posinf=1e-10, neginf=1e-10)

        rr = -invd_sparse(u) @ ns
        r[ps_dense.flatten() == 0] = rr[ps_dense.flatten() == 0]

        # Update s2
        pr = P.T @ r
        nr = N.T @ invd_sparse(r) @ np.ones((m, 1))

        pr_dense = pr.toarray() if sp.issparse(pr) else pr
        nr_dense = nr.toarray() if sp.issparse(nr) else nr
        s2 = invd_sparse(2 * pr_dense) @ (v + np.sqrt(v**2 + 4 * pr_dense * nr_dense))

        # Handle possible NaNs
        s2 = np.nan_to_num(s2, nan=1e-10, posinf=1e-10, neginf=1e-10)

        ss = -invd_sparse(v) @ nr
        s2[pr_dense.flatten() == 0] = ss[pr_dense.flatten() == 0]

        # Convergence check
        dif = np.max(np.abs(s2 - s1))
        s1 = s2  # Update s1 for next iteration

    # Post iteration checks
    if (iter == max_iter) and (dif > EPSILON):
        warnings.warn("GRAS procedure did not converge.")
        return None
    else:
        s = s2
        ps = P @ s
        ns = N @ invd_sparse(s) @ np.ones((n, 1))

        ps_dense = ps.toarray() if sp.issparse(ps) else ps
        ns_dense = ns.toarray() if sp.issparse(ns) else ns
        r = invd_sparse(2 * ps_dense) @ (u + np.sqrt(u**2 + 4 * ps_dense * ns_dense))

        # Handle possible NaNs
        r = np.nan_to_num(r, nan=1e-10, posinf=1e-10, neginf=1e-10)

        rr = -invd_sparse(u) @ ns
        r[ps_dense.flatten() == 0] = rr[ps_dense.flatten() == 0]

        X_GRAS = GRAS_UpdateMatrix(P, N, r, s)
        row_sums = np.array(X_GRAS.sum(axis=1)).flatten()
        column_sums = np.array(X_GRAS.sum(axis=0)).flatten()
        if not (
            np.allclose(row_sums, u.flatten(), atol=epsilon_constr)
            and np.allclose(column_sums, v.flatten(), atol=epsilon_constr)
        ):
            warnings.warn(
                "The final balanced matrix does not match the target row/column sums within the specified tolerance."
            )

        # Final updated matrix
        return X_GRAS, iter, dif

In [15]:
%%time

m, n = (10, 50)

EPSILON = 1e-14

X = np.random.rand(m, n) - 0.5

v, u = X.sum(axis=0), X.sum(axis=1)

v = v.reshape((n, 1))
u = u.reshape((m, 1))

assert np.allclose(u.sum(), v.sum())

u[0, 0] = u[0, 0] + 1
v[0, 0] = v[0, 0] + 1

assert np.allclose(u.sum(), v.sum())

CPU times: total: 0 ns
Wall time: 999 μs


In [16]:
X_new, _, _ = GRAS_Balancing(X, u, v, EPSILON=EPSILON, max_iter=1)

In [19]:
# Test Case 1: Small Basic Matrix
print("Test Case 1: Small Basic Matrix")
X1 = np.array([[5, -3, 0], [2, 0, -2], [-1, 3, 4]])
u1 = np.array([4, 3, 5]).reshape(-1, 1)
v1 = np.array([3, 4, 5]).reshape(-1, 1)
result1, iterations1, diff1 = GRAS_Balancing(X1, u1, v1)

assert result1 is not None, "Test Case 1 failed: Procedure did not converge."
# Check if the sign of each element is preserved
assert np.all((X1 > 0) == (result1 > 0)), (
    "Test Case 1 failed: Signs of positive cells are not preserved."
)
assert np.all((X1 < 0) == (result1 < 0)), (
    "Test Case 1 failed: Signs of negative cells are not preserved."
)
print(f"Test Case 1 passed: Converged in {iterations1} iterations with final difference {diff1}")

Test Case 1: Small Basic Matrix
Test Case 1 passed: Converged in 1 iterations with final difference 7.62496732420459e-11


In [20]:
# Test Case 2: Diagonal Matrix
print("\nTest Case 2: Diagonal Matrix")
X2 = np.array([[5, 0, 0], [0, -4, 0], [0, 0, 3]])
u2 = np.array([5, -4, 3]).reshape(-1, 1)
v2 = np.array([5, -4, 3]).reshape(-1, 1)
result2, iterations2, diff2 = GRAS_Balancing(X2, u2, v2)
assert result2 is not None, "Test Case 2 failed: Procedure did not converge."
# Check if the sign of each element is preserved
assert np.all((X2 > 0) == (result2 > 0)), (
    "Test Case 2 failed: Signs of positive cells are not preserved."
)
assert np.all((X2 < 0) == (result2 < 0)), (
    "Test Case 2 failed: Signs of negative cells are not preserved."
)
print(f"Test Case 2 passed: Converged in {iterations2} iterations with final difference {diff2}")


Test Case 2: Diagonal Matrix
Test Case 2 passed: Converged in 1 iterations with final difference 0.0


In [21]:
# Test Case 3: All Positive Matrix
print("\nTest Case 3: All Positive Matrix")
X3 = np.array([[3, 2, 1], [4, 5, 6], [7, 8, 9]])
u3 = np.array([7, 16, 24]).reshape(-1, 1)
v3 = np.array([21, 15, 11]).reshape(-1, 1)
result3, iterations3, diff3 = GRAS_Balancing(X3, u3, v3)
assert result3 is not None, "Test Case 3 failed: Procedure did not converge."
# Check if the sign of each element is preserved
assert np.all((X3 > 0) == (result3 > 0)), (
    "Test Case 3 failed: Signs of positive cells are not preserved."
)
print(f"Test Case 3 passed: Converged in {iterations3} iterations with final difference {diff3}")


Test Case 3: All Positive Matrix
Test Case 3 passed: Converged in 1 iterations with final difference 9.164669023675742e-11


In [23]:
# Test Case 4: Large Sparse Matrix
print("\nTest Case 4: Large Sparse Matrix")
X4 = sp.random(100, 100, density=0.1, format="csr") - sp.random(100, 100, density=0.1, format="csr")
u4 = np.random.rand(100, 1) * 10
v4 = np.random.rand(100, 1) * 10
u4 = u4 / u4.sum() * v4.sum()  # Normalize u4 to ensure u4.sum() == v4.sum()
result4, iterations4, diff4 = GRAS_Balancing(X4.toarray(), u4, v4)
assert result4 is not None, "Test Case 4 failed: Procedure did not converge."
# Check if the sign of each element is preserved
assert np.all((X4.toarray() > 0) == (result4 > 0)), (
    "Test Case 4 failed: Signs of positive cells are not preserved."
)
assert np.all((X4.toarray() < 0) == (result4 < 0)), (
    "Test Case 4 failed: Signs of negative cells are not preserved."
)
print(f"Test Case 4 passed: Converged in {iterations4} iterations with final difference {diff4}")


Test Case 4: Large Sparse Matrix
Test Case 4 passed: Converged in 1 iterations with final difference 5.643485678774596e-11
